# [Serverless Machine Learning in Action](https://www.manning.com/books/serverless-machine-learning-in-action?a_aid=osipov&a_bid=fa913283&)
## by Carl Osipov

## Source Code for [Chapter 3](https://livebook.manning.com/book/serverless-machine-learning-in-action/chapter-2?a_aid=osipov&a_bid=fa913283&) 

## <font color=red>Upload the `BUCKET_ID` file</font>

Before proceeding, ensure that you have a backup copy of the `BUCKET_ID` file created in the [Chapter 2](https://colab.research.google.com/github/osipov/smlbook/blob/master/ch2.ipynb) notebook before proceeding. The contents of the `BUCKET_ID` file are reused later in this notebook and in the other notebooks.


In [ ]:
import os
from pathlib import Path
assert Path('BUCKET_ID').exists(), "Place the BUCKET_ID file in the current directory before proceeding"

BUCKET_ID = Path('BUCKET_ID').read_text().strip()
os.environ['BUCKET_ID'] = BUCKET_ID
os.environ['BUCKET_ID']

## **OPTIONAL:** Download and install AWS CLI

This is unnecessary if you have already installed AWS CLI in a preceding notebook.

In [ ]:
%%bash
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip -o awscliv2.zip
sudo ./aws/install

## Specify AWS credentials

Modify the contents of the next cell to specify your AWS credentials as strings. 

If you see the following exception:

`TypeError: str expected, not NoneType`

It means that you did not specify the credentials correctly.

In [ ]:
import os
# *** REPLACE None in the next 2 lines with your AWS key values ***
os.environ['AWS_ACCESS_KEY_ID'] = None
os.environ['AWS_SECRET_ACCESS_KEY'] = None

## Confirm the credentials

Run the next cell to validate your credentials.

In [ ]:
%%bash
aws sts get-caller-identity

If you have specified the correct credentials as values for the `AWS_ACCESS_KEY_ID` and the `AWS_SECRET_ACCESS_KEY` environment variables, then `aws sts get-caller-identity` used by the previous cell should have returned back the `UserId`, `Account` and the `Arn` for the credentials, resembling the following

```
{
    "UserId": "█████████████████████",
    "Account": "████████████",
    "Arn": "arn:aws:iam::████████████:user/█████████"
}
```

## Specify the region

Replace the `None` in the next cell with your AWS region name, for example `us-west-2`.

In [ ]:
# *** REPLACE None in the next line with your AWS region ***
os.environ['AWS_DEFAULT_REGION'] = None

If you have specified the region correctly, the following cell should return back the region that you have specifies.

In [ ]:
%%bash
echo $AWS_DEFAULT_REGION

## Download a tiny sample

Download a tiny sample of the dataset from https://gist.github.com/osipov/1fc0265f8f829d9d9eee8393657423a9 to a `trips_sample.csv` file which you are going to use to learn about using the Athena interface.

In [ ]:
%%bash
wget -q https://gist.githubusercontent.com/osipov/1fc0265f8f829d9d9eee8393657423a9/raw/9957c1f09cdfa64f8b8d89cfec532a0e150d5178/trips_sample.csv
ls -ltr trips_sample.csv
cat trips_sample.csv

Assuming the previous cell executed successfully, it should have returned the following result:

```
-rw-r--r-- 1 root root 378 Nov 23 19:50 trips_sample.csv
fareamount_double,origin_block_latitude,origin_block_longitude,destination_block_latitude,destination_block_longitude
8.11,38.900769,-77.033644,38.912239,-77.036514
5.95,38.912609,-77.030788,38.906445,-77.023978
7.57,38.900773,-77.03655,38.896131,-77.024975
11.61,38.892101000000004,-77.044208,38.905969,-77.06564399999999
4.87,38.899615000000004,-76.980387,38.900638,-76.97023
```

## Upload `trips_sample.csv` to your object storage bucket

In [ ]:
%%bash
aws s3 cp trips_sample.csv s3://dc-taxi-$BUCKET_ID-$AWS_DEFAULT_REGION/samples/trips_sample.csv
aws s3 ls s3://dc-taxi-$BUCKET_ID-$AWS_DEFAULT_REGION/samples/trips_sample.csv

The output of the `aws s3 ls` command above should include the following, confirming that the file was uploaded successfully.

```
2020-11-23 20:07:31        378 trips_sample.csv
```

## Create an Athena workgroup

Create a `dc_taxi_athena_workgroup` for your Athena project, assuming one does not exist yet.

In [ ]:
%%bash
aws athena create-work-group --name dc_taxi_athena_workgroup \
--configuration "ResultConfiguration={OutputLocation=s3://dc-taxi-$BUCKET_ID-$AWS_DEFAULT_REGION/athena},EnforceWorkGroupConfiguration=false,PublishCloudWatchMetricsEnabled=false"

## Query Athena and Report Query Status

In [ ]:
%%bash
SQL="
CREATE EXTERNAL TABLE IF NOT EXISTS dc_taxi_db.dc_taxi_csv_sample_strings(
        fareamount STRING,
        origin_block_latitude STRING,
        origin_block_longitude STRING,
        destination_block_latitude STRING,
        destination_block_longitude STRING
)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
LOCATION 's3://dc-taxi-$BUCKET_ID-$AWS_DEFAULT_REGION/samples/'
TBLPROPERTIES ('skip.header.line.count'='1');"

ATHENA_QUERY_ID=$(aws athena start-query-execution \
--work-group dc_taxi_athena_workgroup \
--query 'QueryExecutionId' \
--output text \
--query-string "$SQL")

echo $SQL

echo $ATHENA_QUERY_ID
until aws athena get-query-execution \
  --query 'QueryExecution.Status.State' \
  --output text \
  --query-execution-id $ATHENA_QUERY_ID | grep -v "RUNNING";
do
  printf '.'
  sleep 1; 
done

## Create a Script to Query Athena 

In [ ]:
%%writefile utils.sh
#!/bin/bash 
function athena_query {
    local SQL=$1
    local ATHENA_QUERY_ID=$(aws athena start-query-execution \
                            --work-group dc_taxi_athena_workgroup \
                            --query 'QueryExecutionId' \
                            --output text \
                            --query-string "$SQL")
    echo $ATHENA_QUERY_ID
}
function athena_query_to_table {
    local SQL=$1
    local OUTPUT_QUERY=$2
    local MAX_ITEMS=${3:-10}

    local ATHENA_QUERY_ID=$(athena_query "$SQL")
    echo $ATHENA_QUERY_ID
    until aws athena get-query-execution \
      --query 'QueryExecution.Status.State' \
      --output text \
      --query-execution-id $ATHENA_QUERY_ID | grep -v "RUNNING";
    do
      printf '.'
      sleep 1; 
    done

    aws athena get-query-results \
      --max-items $MAX_ITEMS \
      --query-execution-id $ATHENA_QUERY_ID \
      --query $OUTPUT_QUERY \
      --output table
}

function athena_query_to_pandas {
    local SQL=$1
    local OUTPUT_QUERY=${2:-"ResultSet.Rows[*].[Data[].VarCharValue]"}
    local MAX_ITEMS=${3:-10}
    local TMP_JSON=${4:-"/tmp/awscli.json"}

    local ATHENA_QUERY_ID=$(athena_query "$SQL")
    echo $ATHENA_QUERY_ID
    until aws athena get-query-execution \
      --query 'QueryExecution.Status.State' \
      --output text \
      --query-execution-id $ATHENA_QUERY_ID | grep -v "RUNNING";
    do
      printf '.'
      sleep 1; 
    done

    aws athena get-query-results \
      --max-items $MAX_ITEMS \
      --query-execution-id $ATHENA_QUERY_ID \
      --query $OUTPUT_QUERY \
      --output json > $TMP_JSON      
}

## Output Athena Query to a Text Table

In [ ]:
%%bash
source utils.sh
SQL="
SELECT

origin_block_latitude || ' , ' || origin_block_longitude
    AS origin,

destination_block_latitude || '  , ' || destination_block_longitude
    AS destination

FROM
    dc_taxi_db.dc_taxi_csv_sample_strings
"
athena_query_to_table "$SQL" "ResultSet.Rows[*].[Data[0].VarCharValue,Data[1].VarCharValue]"

## Output Athena Query to JSON for a Pandas DataFrame

In [ ]:
%%bash
source utils.sh ; athena_query_to_pandas """
SELECT

origin_block_latitude || ' , ' || origin_block_longitude
    AS origin,

destination_block_latitude || '  , ' || destination_block_longitude
    AS destination

FROM
    dc_taxi_db.dc_taxi_csv_sample_strings
"""

## Create a Utility Function to Read AWS CLI JSON as Pandas

In [ ]:
import pandas as pd
def awscli_to_df():
  json_df = pd.read_json('/tmp/awscli.json')
  df = pd.DataFrame(json_df[0].tolist(), index = json_df.index, columns = json_df[0].tolist()[0]).drop(0, axis = 0)
  return df

In [ ]:
awscli_to_df()

## Apply Athena schema-on-read with columns as `DOUBLE`

In [ ]:
%%bash
source utils.sh ; athena_query "
CREATE EXTERNAL TABLE IF NOT EXISTS dc_taxi_db.dc_taxi_csv_sample_double(
        fareamount DOUBLE,
        origin_block_latitude DOUBLE,
        origin_block_longitude DOUBLE,
        destination_block_latitude DOUBLE,
        destination_block_longitude DOUBLE
)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
LOCATION 's3://dc-taxi-$BUCKET_ID-$AWS_DEFAULT_REGION/samples/'
TBLPROPERTIES ('skip.header.line.count'='1');
"

In [ ]:
%%bash
source utils.sh ; athena_query_to_pandas "
SELECT ROUND(MAX(fareamount) - MIN(fareamount), 2)
FROM dc_taxi_db.dc_taxi_csv_sample_double
"

In [ ]:
awscli_to_df()

## Explore 10 records from the DC taxi dataset

In [ ]:
%%bash
source utils.sh ; athena_query_to_pandas "
SELECT fareamount_double,
         origindatetime_tr,
         origin_block_latitude,
         origin_block_longitude,
         destination_block_latitude,
         destination_block_longitude
FROM dc_taxi_db.dc_taxi_parquet
LIMIT 10
"

In [ ]:
awscli_to_df()

## What is the number of the timestamps with NULL values?

In [ ]:
%%bash
source utils.sh ; athena_query_to_pandas "
SELECT
    COUNT(*)
      AS total
FROM
    dc_taxi_db.dc_taxi_parquet
WHERE
    origindatetime_tr IS NULL
"

In [ ]:
awscli_to_df()

## How many timestamps are un-parsable?

In [ ]:
%%bash
source utils.sh ; athena_query_to_pandas "
SELECT
    (SELECT COUNT(*) FROM dc_taxi_db.dc_taxi_parquet)
    - COUNT(DATE_PARSE(origindatetime_tr, '%m/%d/%Y %H:%i'))
    AS origindatetime_not_parsed
FROM
    dc_taxi_db.dc_taxi_parquet
WHERE
    origindatetime_tr IS NOT NULL;
"

In [ ]:
awscli_to_df()

## How often are parts of the pick up location coordinate missing?

In [ ]:
%%bash
source utils.sh ; athena_query_to_pandas "
SELECT
    ROUND(100.0*COUNT(*) / (SELECT COUNT(*)
                        FROM dc_taxi_db.dc_taxi_parquet), 2)

        AS percentage_null,

    (SELECT COUNT(*)
     FROM dc_taxi_db.dc_taxi_parquet
     WHERE origin_block_longitude IS NULL
     OR origin_block_latitude IS NULL)

        AS either_null,

    (SELECT COUNT(*)
     FROM dc_taxi_db.dc_taxi_parquet
     WHERE origin_block_longitude IS NULL
     AND origin_block_latitude IS NULL)

        AS both_null

FROM
    dc_taxi_db.dc_taxi_parquet
WHERE
    origin_block_longitude IS NULL
    OR origin_block_latitude IS NULL
GROUP BY
    origin_block_longitude, origin_block_latitude
"

In [ ]:
awscli_to_df()

## How often are parts of the drop off coordinates missing?

Repeat the previous analysis

In [ ]:
%%bash
source utils.sh ; athena_query_to_pandas "
SELECT
    ROUND(100.0*COUNT(*) / (SELECT COUNT(*)
                        FROM dc_taxi_db.dc_taxi_parquet), 2)

        AS percentage_null,

    (SELECT COUNT(*)
     FROM dc_taxi_db.dc_taxi_parquet
     WHERE destination_block_longitude IS NULL
     OR destination_block_latitude IS NULL)

        AS either_null,

    (SELECT COUNT(*)
     FROM dc_taxi_db.dc_taxi_parquet
     WHERE destination_block_longitude IS NULL
     AND destination_block_latitude IS NULL)

        AS both_null

FROM
    dc_taxi_db.dc_taxi_parquet
WHERE
    destination_block_longitude IS NULL
    OR destination_block_latitude IS NULL
GROUP BY
    destination_block_longitude, destination_block_latitude
"

In [ ]:
awscli_to_df()

 ## Find the count and the fraction of the missing coordinates

In [ ]:
%%bash
source utils.sh ; athena_query_to_pandas "
SELECT
    COUNT(*)
      AS total,

    ROUND(100.0 * COUNT(*) / (SELECT COUNT(*)
                              FROM dc_taxi_db.dc_taxi_parquet), 2)
      AS percent

FROM
    dc_taxi_db.dc_taxi_parquet

WHERE
    origin_block_latitude IS NULL
    OR origin_block_longitude IS NULL
    OR destination_block_latitude IS NULL
    OR destination_block_longitude IS NULL
"

In [ ]:
awscli_to_df()

## Query for the values and quantities of fareamount_string that failed to parse as a double

In [ ]:
%%bash
source utils.sh ; athena_query_to_pandas "
SELECT
    fareamount_string,
    COUNT(fareamount_string) AS rows,
    ROUND(100.0 * COUNT(fareamount_string) /
          ( SELECT COUNT(*)
            FROM dc_taxi_db.dc_taxi_parquet), 2)

      AS percent

FROM
    dc_taxi_db.dc_taxi_parquet
WHERE
    fareamount_double IS NULL
    AND fareamount_string IS NOT NULL
GROUP BY
    fareamount_string;
"

In [ ]:
awscli_to_df()

## Explore summary statistics of the `fareamount_double` column

In [ ]:
%%bash
source utils.sh ; athena_query_to_pandas "
WITH

src AS (SELECT
            fareamount_double AS val
        FROM
            dc_taxi_db.dc_taxi_parquet),

stats AS
    (SELECT
        MIN(val) AS min,
        APPROX_PERCENTILE(val, 0.25) AS q1,
        APPROX_PERCENTILE(val ,0.5) AS q2,
        APPROX_PERCENTILE(val, 0.75) AS q3,
        AVG(val) AS mean,
        STDDEV(val) AS std,
        MAX(val) AS max
    FROM
        src)

SELECT
    DISTINCT min, q1, q2, q3, max

FROM
    dc_taxi_db.dc_taxi_parquet, stats
"

In [ ]:
awscli_to_df()

## What percentage of fare amount values are null or below the minimum threshold?

In [ ]:
%%bash
source utils.sh ; athena_query_to_pandas "
WITH
src AS (SELECT
            COUNT(*) AS total
        FROM
            dc_taxi_db.dc_taxi_parquet
        WHERE
            fareamount_double IS NOT NULL)

SELECT
    ROUND(100.0*COUNT(fareamount_double) / MIN(total), 2) AS percent
FROM
    dc_taxi_db.dc_taxi_parquet, src
WHERE
    fareamount_double < 3.25
    AND fareamount_double IS NOT NULL
"

In [ ]:
awscli_to_df()

## Compute summary statistics for the cases where the fareamount_string failed to parse

In [ ]:
%%bash
source utils.sh ; athena_query_to_pandas "
SELECT
    fareamount_string,
    ROUND( MIN(mileage_double), 2) AS min,
    ROUND( APPROX_PERCENTILE(mileage_double, 0.25), 2) AS q1,
    ROUND( APPROX_PERCENTILE(mileage_double ,0.5), 2) AS q2,
    ROUND( APPROX_PERCENTILE(mileage_double, 0.75), 2) AS q3,
    ROUND( MAX(mileage_double), 2) AS max
FROM
    dc_taxi_db.dc_taxi_parquet
WHERE
    fareamount_string LIKE 'NULL'
GROUP BY
    fareamount_string
"

In [ ]:
awscli_to_df()

## Figure out the lower left and upper right boundary locations

Plugging the latitude and longitude coordinates reported by the query into OpenStreetMap ( https://www.openstreetmap.org/directions?engine=fossgis_osrm_car&route=38.8110%2C-77.1130%3B38.9950%2C-76.9100#map=11/38.9025/-77.0094 ) yields 21.13 miles or an estimate of $ \$48.89 (21.13 * \$2.16/mile + \$3.25) $.


In [ ]:
%%bash
source utils.sh ; athena_query_to_pandas "
SELECT
    MIN(origin_block_latitude) AS lower_left_latitude,
    MIN(origin_block_longitude) AS lower_left_longitude,
    MAX(destination_block_latitude) AS upper_right_latitude,
    MAX(destination_block_longitude) AS upper_right_longitude
FROM
    dc_taxi_db.dc_taxi_parquet;
"

In [ ]:
awscli_to_df()

## Compute normally distributed averages of random samples

In [ ]:
%%bash
source utils.sh ; athena_query_to_pandas "
WITH dc_taxi AS 
(SELECT *, 
    origindatetime_tr 
    || fareamount_string 
    || origin_block_latitude_string 
    || origin_block_longitude_string 
    || destination_block_latitude_string 
    || destination_block_longitude_string 
    || mileage_string AS objectid

    FROM "dc_taxi_db"."dc_taxi_parquet"

    WHERE fareamount_double >= 3.25
            AND fareamount_double IS NOT NULL
            AND mileage_double > 0 )

SELECT AVG(mileage_double) AS average_mileage
FROM dc_taxi
WHERE objectid IS NOT NULL
GROUP BY  MOD( ABS( from_big_endian_64( xxhash64( to_utf8( objectid ) ) ) ), 1000)
" ResultSet.Rows[*].[Data[].VarCharValue] 1000

In [ ]:
awscli_to_df()

## Visually confirm that the means of samples are normally distributed

In [ ]:
%matplotlib inline
df = awscli_to_df()
df.average_mileage = df.average_mileage.astype(float)
df.average_mileage -= df.average_mileage.mean()
df.average_mileage /= df.average_mileage.std()
df.average_mileage.plot.hist(bins = 30);

## Produce a statistical upper bound estimate for the mileage

In [ ]:
%%bash
source utils.sh ; athena_query_to_pandas "
WITH dc_taxi AS 
(SELECT *, 
    origindatetime_tr 
    || fareamount_string 
    || origin_block_latitude_string 
    || origin_block_longitude_string 
    || destination_block_latitude_string 
    || destination_block_longitude_string 
    || mileage_string AS objectid

    FROM "dc_taxi_db"."dc_taxi_parquet"

    WHERE fareamount_double >= 3.25
            AND fareamount_double IS NOT NULL
            AND mileage_double > 0 ),

dc_taxi_samples AS (
    SELECT AVG(mileage_double) AS average_mileage
    FROM dc_taxi
    WHERE objectid IS NOT NULL
    GROUP BY  MOD( ABS( from_big_endian_64( xxhash64( to_utf8( objectid ) ) ) ) , 1000)
)
SELECT AVG(average_mileage) + 4 * STDDEV(average_mileage)
FROM dc_taxi_samples
"

In [ ]:
awscli_to_df()

In [ ]:
upper_mileage = 2.16 * awscli_to_df().mean().item() + 3.25
upper_mileage

## Produce the final estimate for the upper bound on fare amount

In [ ]:
%%bash
source utils.sh ; athena_query_to_pandas "
WITH dc_taxi AS 
(SELECT *, 
    origindatetime_tr 
    || fareamount_string 
    || origin_block_latitude_string 
    || origin_block_longitude_string 
    || destination_block_latitude_string 
    || destination_block_longitude_string 
    || mileage_string AS objectid

    FROM "dc_taxi_db"."dc_taxi_parquet"

    WHERE fareamount_double >= 3.25
            AND fareamount_double IS NOT NULL
            AND mileage_double > 0 ),

dc_taxi_samples AS (
    SELECT AVG(fareamount_double) AS average_fareamount
    FROM dc_taxi
    WHERE objectid IS NOT NULL
    GROUP BY  MOD( ABS( from_big_endian_64( xxhash64( to_utf8( objectid ) ) ) ) , 1000)
)
SELECT AVG(average_fareamount) + 4 * STDDEV(average_fareamount)
FROM dc_taxi_samples
"

In [ ]:
awscli_to_df()

In [ ]:
upper_fareamount = awscli_to_df().mean().item()
upper_fareamount

In [ ]:
means = [15.96, 29.19, 48.89, 560, 2,422.45]
sum(means) / len(means)

## Check the percentage of the dataset above the upper fare amount bound

In [ ]:
%%bash
source utils.sh ; athena_query_to_pandas "
SELECT
    100.0*COUNT(fareamount_double) / (SELECT COUNT(*) FROM dc_taxi_db.dc_taxi_parquet WHERE fareamount_double IS NOT NULL) AS percent
FROM
    dc_taxi_db.dc_taxi_parquet
WHERE (fareamount_double < 3.25 OR fareamount_double > 179.75)
        AND fareamount_double IS NOT NULL;
"

In [ ]:
awscli_to_df()

## Produce final summary statistics

In [ ]:
%%bash
source utils.sh ; athena_query_to_pandas "
WITH src AS (SELECT fareamount_double AS val
             FROM dc_taxi_db.dc_taxi_parquet
             WHERE fareamount_double IS NOT NULL
             AND fareamount_double >= 3.25
             AND fareamount_double <= 180.0),
stats AS
    (SELECT
     ROUND(MIN(val), 2) AS min,
     ROUND(APPROX_PERCENTILE(val, 0.25), 2) AS q1,
     ROUND(APPROX_PERCENTILE(val, 0.5), 2) AS q2,
     ROUND(APPROX_PERCENTILE(val, 0.75), 2) AS q3,
     ROUND(AVG(val), 2) AS mean,
     ROUND(STDDEV(val), 2) AS std,
     ROUND(MAX(val), 2) AS max
    FROM src)
SELECT min, q1, q2, q3, max, mean, std
FROM stats;
"

In [ ]:
awscli_to_df()

## Check that minimum and maximum locations are within DC boundaries

Using the SQL query and OpenStreetMap ( https://www.openstreetmap.org/directions?engine=fossgis_osrm_car&route=38.8106%2C-77.1134%3B38.9940%2C-76.9100#map=11/38.9025/-77.0210 )  check that the minimum and maximum coordinates for the origin latitude and longitude columns confirm that resulting pairs (38.81138, -77.113633) and (38.994217, -76.910012) are within DC boundaries.





In [ ]:
%%bash
source utils.sh ; athena_query_to_pandas "
SELECT
    MIN(destination_block_latitude),
    MIN(destination_block_longitude),
    MAX(destination_block_latitude),
    MAX(destination_block_longitude),
    MIN(origin_block_latitude),
    MIN(origin_block_longitude),
    MAX(origin_block_latitude),
    MAX(origin_block_longitude)

FROM
    dc_taxi_db.dc_taxi_parquet
"

In [ ]:
awscli_to_df()

Copyright 2020 CounterFactual.AI LLC. All Rights Reserved.

Licensed under the GNU General Public License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. 

You may obtain a copy of the License at

https://github.com/osipov/smlbook/blob/master/LICENSE

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.